In [ ]:
'''
This notebook compare the data efficiency of baseline RBF with known pendulum invariance model
'''
import gpflow
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from scipy.integrate import solve_ivp, odeint
from gpflow.utilities import print_summary, positive, to_default_float, set_trainable
from invariance_kernels import *
from invariance_functions import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
'''
experiment setup
'''
time_step = 0.01
training_time = 0.12
testing_time = 1

max_x = 150
max_v = 30


In [ ]:
'''
fit the model with different number of data points and evaluate their performance
'''
def energy(X):
    return 0.5*tf.square(X[:,1])-np.cos(X[:,0])

mean = ZeroMean(2)
data_eff_moi = []
data_eff_moi_trac = []

test_starting_position = np.radians(np.random.uniform(-max_x, max_x))
test_starting_velocity = np.radians(np.random.uniform(-max_v, max_v))

test_starting = (test_starting_position, test_starting_velocity)
time_setting = (testing_time, time_step)

for n_train in [1, 2, 5, 10]:
    train_starting_position = np.random.uniform(-max_x, max_x, (n_train))
    train_starting_velocity = np.random.uniform(-max_v, max_v, (n_train))

    print(train_starting_position)
    print(train_starting_velocity)
    data = get_pendulum_data(time_step, training_time, 1e-8, train_starting_position, train_starting_velocity) #switch

    moi, *dump = get_GPR_model(get_MOI(), mean, data, 500)
    print("moi:"+format(moi.log_marginal_likelihood().numpy(),".2f"))
    jitter = 5e-4
    invar_density = 40
    invar_range = 3
    dynamics = pendulum_dynamics 

    if n_train == 1:
        kernel = get_invariance_kernel(PendulumInvariance, invar_range, invar_density, jitter) #switch
        known, *dump = get_GPR_model(kernel, mean, data, 500)
        evaluate_invariance = evaluate_model_future(known, test_starting, dynamics,time_setting, energy)
        data_eff_inv = evaluate_invariance[0]

    evaluate_moi = evaluate_model_future(moi, test_starting, dynamics,time_setting, energy)


    data_eff_moi.append(evaluate_moi[0])
    data_eff_moi_trac.append(evaluate_moi)

    


In [ ]:
'''
plot the result
'''
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
plt.plot(evaluate_moi[4][:,0],evaluate_moi[4][:,1], label="ground truth", color="black", linewidth=2)
#plt.plot(evaluate_moi[1][:,0],evaluate_moi[1][:,1], "--", label='RBF', color="red")
plt.plot(data_eff_moi_trac[0][1][:,0],data_eff_moi_trac[0][1][:,1], "--", label='RBF 10 points')
plt.plot(data_eff_moi_trac[1][1][:,0],data_eff_moi_trac[1][1][:,1], "--", label='RBF 20 points')
plt.plot(data_eff_moi_trac[2][1][:,0],data_eff_moi_trac[2][1][:,1], "--", label='RBF 50 points')
plt.plot(data_eff_moi_trac[3][1][:,0],data_eff_moi_trac[3][1][:,1], "--", label='RBF 100 points')
plt.plot(evaluate_invariance[1][:,0],evaluate_invariance[1][:,1], "--", label="invariance 10 points")
plt.scatter(evaluate_moi[4][0,0],evaluate_moi[4][0,1],color="red")
plt.xlabel("q")
plt.ylabel("p")
plt.legend()
plt.savefig("figures/data_efficiency.pdf")